In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run parse.ipynb

In [2]:
# alright, let's try with one movement, one performer!
X = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))
y = torch.Tensor(parseGroundtruthFile('../bach-violin-dataset/dataset/ground-truth/emil-telmanyi/bwv1002/emil-telmanyi_bwv1002_mov1.csv'))

In [3]:
# never mind, let's try combining all the inputs.....?

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.actfn = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.actfn(x)
        x = self.linear2(x)
        return x

In [5]:
model = MLP(3, 4, 2)

In [6]:
# Just in case our model is retaining bad parameters
# model.reset_parameters()

In [7]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [8]:
# Template code from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#The-Basics-of-PyTorch
def train_model(model, optimizer, input_notes, truth, loss_module, num_epochs=100):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        # data_inputs = data_inputs.to(device)
        # data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)
        print(preds)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, y)
        print(loss)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [9]:
train_model(model, optimizer, X, y, loss)

tensor([[  12.8263,    6.2348],
        [  10.3090,    2.7486],
        [  11.4635,    3.3638],
        ...,
        [ 245.9501, -336.8405],
        [ 247.2116, -338.1453],
        [ 248.6307, -339.6133]], grad_fn=<AddmmBackward0>)
tensor(49971.8633, grad_fn=<MseLossBackward0>)
tensor([[  11.3649,   19.6917],
        [   8.7834,   18.1582],
        [   9.8165,   19.8639],
        ...,
        [-178.0975,  941.6714],
        [-177.0010,  941.6237],
        [-175.7675,  941.5700]], grad_fn=<AddmmBackward0>)
tensor(120298.8203, grad_fn=<MseLossBackward0>)
tensor([[ 1.8396e+01,  3.8768e+00],
        [ 1.5296e+01,  3.0031e-01],
        [ 1.6921e+01,  7.4006e-01],
        ...,
        [ 3.4536e+02, -3.1791e+02],
        [ 3.4624e+02, -3.1872e+02],
        [ 3.4724e+02, -3.1963e+02]], grad_fn=<AddmmBackward0>)
tensor(49848.9492, grad_fn=<MseLossBackward0>)
tensor([[ 15.8250,  11.1852],
        [ 12.1741,   7.3608],
        [ 13.5934,   8.4234],
        ...,
        [126.9986, -19.7883],
     

In [10]:
X_test = torch.Tensor(parseInputFile('../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov3.csv'))
y_test = torch.Tensor.tolist(model(X_test))

X_test = torch.Tensor.tolist(X_test)
print(X_test)
for i in range(len(y_test)):
    y_test[i].append(X_test[i][2])

print(y_test)

[[0.0, 18.0, 58.0], [18.0, 24.0, 62.0], [24.0, 36.0, 65.0], [36.0, 42.0, 63.0], [42.0, 48.0, 62.0], [48.0, 54.0, 60.0], [54.0, 60.0, 58.0], [60.0, 72.0, 74.0], [60.0, 72.0, 82.0], [72.0, 84.0, 65.0], [72.0, 84.0, 74.0], [72.0, 84.0, 82.0], [84.0, 96.0, 72.0], [84.0, 96.0, 81.0], [96.0, 108.0, 70.0], [96.0, 108.0, 79.0], [108.0, 120.0, 70.0], [108.0, 120.0, 79.0], [120.0, 132.0, 69.0], [120.0, 132.0, 77.0], [132.0, 144.0, 63.0], [144.0, 156.0, 62.0], [156.0, 168.0, 72.0], [156.0, 168.0, 77.0], [168.0, 180.0, 70.0], [168.0, 180.0, 79.0], [180.0, 192.0, 60.0], [192.0, 198.0, 75.0], [192.0, 204.0, 69.0], [198.0, 204.0, 72.0], [204.0, 216.0, 58.0], [204.0, 216.0, 70.0], [204.0, 216.0, 74.0], [216.0, 228.0, 65.0], [228.0, 234.0, 74.0], [228.0, 246.0, 70.0], [234.0, 240.0, 77.0], [240.0, 246.0, 75.0], [246.0, 252.0, 70.0], [246.0, 252.0, 74.0], [252.0, 264.0, 69.0], [252.0, 264.0, 72.0], [264.0, 276.0, 57.0], [276.0, 288.0, 60.0], [288.0, 300.0, 65.0], [300.0, 306.0, 69.0], [300.0, 306.0, 72.